In [1]:
import sys
import os

# Source: https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris

from lusi.ecoc import SVMRandomInvariantsECOC
from lusi.types import InvariantTypes

In [2]:
data = load_iris()

In [3]:
X, y = data.data, data.target

In [4]:
num_dimensions = X.shape[1]
num_classes = len(np.unique(y))

print("Num dimensions: ", num_dimensions)
print("Num classes: ", num_classes)

Num dimensions:  4
Num classes:  3


In [5]:
def run_single_experiment(X_train, X_test, y_train, y_test, clf, model_parameters):
    model = GridSearchCV(clf, model_parameters, cv=5, scoring='accuracy', n_jobs=4)
    model.fit(X_train, y_train)
    
    print('Best estimator: ', model.best_estimator_)
    
    y_hat = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_hat)
    
    print('Accuracy: ', accuracy)
    
    return accuracy

## Experiment 1: Using 80% of the data

### Baseline model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=47)

In [7]:
# Define hyperparameters that will be used in the Grid Search
baseline_parameters = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'gamma': [0.01, 0.1, 1.0, 'auto', 'scale'],
    'degree': np.arange(10),
}

In [8]:
svm_baseline = SVC(random_state=47)

In [9]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

Best estimator:  SVC(C=0.1, degree=2, gamma=1.0, kernel='poly', random_state=47)
Accuracy:  1.0


1.0

### Random projections ECOC

In [10]:
encoding = np.eye(num_classes)
encoding

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [11]:
ecoc_proj_params = {
    'kernel': ['rbf'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
    'delta': [0.001, 0.01, 0.1, 1.0],
    'num_invariants': np.arange(1, num_dimensions + 1),
    'normalize_projections': [False, True],
}

In [12]:
ecoc_projections = SVMRandomInvariantsECOC(
    encoding,
    invariant_type=InvariantTypes.PROJECTION,
    random_state=47,
)

In [13]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.1, delta=0.01,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.1, num_invariants=3, random_state=47)
Accuracy:  1.0


1.0

### Random hyperplanes ECOC

In [14]:
ecoc_hyper_params = {
    'kernel': ['rbf'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
    'delta': [0.001, 0.01, 0.1, 1.0],
    'num_invariants': np.arange(1, num_dimensions + 1),
}

In [15]:
ecoc_hyperplanes = SVMRandomInvariantsECOC(
    encoding,
    invariant_type=InvariantTypes.HYPERPLANE,
    random_state=47,
)

In [16]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.1, delta=0.1,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=4, random_state=47)
Accuracy:  1.0


1.0

## Experiment 2: Using 50% of the data

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=47)

In [18]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

Best estimator:  SVC(degree=0, gamma=1.0, random_state=47)
Accuracy:  0.96


0.96

In [19]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.1, num_invariants=2, random_state=47)
Accuracy:  0.9733333333333334


0.9733333333333334

In [20]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=1, random_state=47)
Accuracy:  0.9466666666666667


0.9466666666666667

## Experiment 3: Using 30% of the data

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=47)

In [22]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

Best estimator:  SVC(C=0.01, degree=4, kernel='poly', random_state=47)
Accuracy:  0.9428571428571428


0.9428571428571428

In [23]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.01, num_invariants=1, random_state=47)
Accuracy:  0.9714285714285714


0.9714285714285714

In [24]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

Best estimator:  SVMRandomInvariantsECOC(C=1.0,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=3, random_state=47)
Accuracy:  0.8476190476190476


0.8476190476190476

## Experiment 4: Using 20% of the data

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=47)

In [26]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

Best estimator:  SVC(C=0.01, degree=2, gamma=1.0, kernel='poly', random_state=47)
Accuracy:  0.9666666666666667


0.9666666666666667

In [27]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.01, num_invariants=1, random_state=47)
Accuracy:  0.9583333333333334


0.9583333333333334

In [28]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

Best estimator:  SVMRandomInvariantsECOC(C=0.001, delta=1.0,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.01,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=1, random_state=47)
Accuracy:  0.9083333333333333


0.9083333333333333

## Experiment 5: Using 10% of the data

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=47)

In [30]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    svm_baseline,
    baseline_parameters
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVC(C=0.001, degree=5, gamma=0.1, kernel='poly', random_state=47)
Accuracy:  0.9407407407407408


0.9407407407407408

In [31]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_projections,
    ecoc_proj_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001, delta=1.0,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.001, normalize_projections=True,
                        num_invariants=3, random_state=47)
Accuracy:  0.9111111111111111


0.9111111111111111

In [32]:
run_single_experiment(
    X_train,
    X_test,
    y_train,
    y_test,
    ecoc_hyperplanes,
    ecoc_hyper_params
)

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]),
                        gamma=0.001,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=1, random_state=47)
Accuracy:  0.9481481481481482


0.9481481481481482